### Generating Medical Imaging Reports

In [123]:
import json
import pydicom
from datetime import datetime

## Writing into DICOM Images

In [78]:
#load selected dicom files to update dicom tags as listed below
ds = pydicom.dcmread("CXR147_IM-0303-1001.dcm") 

In [79]:
#updating values of dicom data
#Patient Name - field exists
ds.PatientName = 'Mark Aldrin'
print(ds.PatientName)
#DOB
print(ds.PatientBirthDate)

#Physician
ds.ReferringPhysicianName = 'Dr. Zeinmann'
print(ds.ReferringPhysicianName)
#Type of Imaging Study
print(ds.ImageType)
#Date Time of Study
print(ds.StudyDate,ds.StudyTime)
#Clinical History (0040,100A)	SQ	Reason for Requested Procedure Code Sequence
ds.add_new(0x00401002,'LO','not presented in dicom file')
#ds.ReasonForRequestedProcedure = 
#Comparison
#Technique
#Findings from model
fin = "The lungs are clear. Heart and pulmonary XXXX appear normal. The pleural spaces are clear and mediastinal contours are normal. Nodular density overlying the anterior left 4th rib XXXX represents a healing rib fracture."
print(fin)
#impressions from model
imp = "1. No acute cardiopulmonary disease"
print(imp)
#tags from model
tags = "normal"
#Body Part Examined
ds.BodyPartExamined = "Chest"
print(ds.BodyPartExamined)
pydicom.write_file('CXR147_IM-0303-1001.dcm',ds)

Mark Aldrin
19391114
Dr. Zeinmann
['DERIVED', 'SECONDARY', '', 'CSA RESAMPLED']
20111121 
The lungs are clear. Heart and pulmonary XXXX appear normal. The pleural spaces are clear and mediastinal contours are normal. Nodular density overlying the anterior left 4th rib XXXX represents a healing rib fracture.
1. No acute cardiopulmonary disease
Chest


In [88]:
from fpdf import FPDF

#pending - include header and footer. Include AGMIR logo
def generate_medical_report(image_file,dcm_file_data):
    pdf = FPDF(format='letter', unit='in')

    effective_page_width = pdf.w - 2*pdf.l_margin
    pdf.add_page()

    #Header
    pdf.set_font('Times','B',14.0)
    pdf.cell(1.0, 0.0,"Radiology Report",ln=1, align="C")
    pdf.ln(0.5)
    
    #Patient Demographic
    pdf.set_font('Times','',10.0) 
    pdf.cell(1.0, 0.0, "Patient Name: "+str(dcm_file_data['PatientName']), ln=1, align="L")
    pdf.ln(0.25)
    
    pdf.cell(1.0, 0.0, txt="Date of Birth: "+str(dcm_file_data['DOB']), ln=1, align="L")
    pdf.ln(0.25)
    
    pdf.cell(1.0, 0.0, txt="Physician: "+str(dcm_file_data['Physician']), ln=1, align="L")
    pdf.ln(0.25)
    
    pdf.cell(1.0, 0.0, txt="Exam: "+str(dcm_file_data['Exam']), ln=1, align="L")
    pdf.ln(0.25)
    
    pdf.cell(1.0, 0.0, txt="Date: "+str(dcm_file_data['ExamDate']), ln=1, align="L")
    pdf.ln(0.5)
    
    #Clinical Information
    pdf.set_font('Times','B',10.0)
    pdf.cell(1.0, 0.0, txt="Clinical History ", ln=1, align="L")
    pdf.set_font('Times','',10.0)
    Clinical_History = str(dcm_file_data['CH'])
    pdf.ln(0.25)
    pdf.multi_cell(effective_page_width*0.8, .15,Clinical_History)
    pdf.ln(0.25)
    
    #Comparison
    #pdf.set_font('Times','B',10.0)
    #pdf.cell(1.0, 0.0, txt="Comparison", ln=1, align="L")
    #Comparison = str(dcm_file_data['Comp'])
    #pdf.ln(0.25)
    #pdf.set_font('Times','',10.0)
    #pdf.multi_cell(effective_page_width, .15,Comparison)
    #pdf.ln(0.25)
    
    #Technique
    #pdf.set_font('Times','B',10.0)
    #pdf.cell(1.0, 0.0, txt="Technique", ln=1, align="L")
    #pdf.ln(0.25)
    #Technique = str(dcm_file_data['Technique'])
    #pdf.set_font('Times','',10.0)
    #pdf.multi_cell(effective_page_width, .15,Technique)
    #pdf.ln(0.25)
    
    #Findings
    pdf.set_font('Times','BI',10.0)
    pdf.cell(1.0, 0.0, txt="Findings", ln=1, align="L")
    pdf.ln(0.25)
    Findings = str(dcm_file_data['Findings'])
    pdf.set_font('Times','',10.0)
    pdf.multi_cell(effective_page_width, .15,Findings)
    pdf.ln(0.25)
    
    #Impression
    pdf.set_font('Times','B',10.0)
    pdf.cell(1.0, 0.0, txt="Impression", ln=1, align="L")
    pdf.ln(0.25)
    Impression = str(dcm_file_data['Impression'])
    pdf.set_font('Times','',10.0)
    pdf.multi_cell(effective_page_width, .15,Impression)
    pdf.ln(0.25)
    
    #Tags
    pdf.set_font('Times','B',10.0)
    pdf.cell(1.0, 0.0, txt="Tags", ln=1, align="L")
    pdf.ln(0.25)
    Tags = str(dcm_file_data['Tags'])
    pdf.set_font('Times','',10.0)
    pdf.multi_cell(effective_page_width, .15,Tags)
    pdf.ln(0.25)
    filename = str(image_file) +'.pdf'
    pdf.output(filename)

In [125]:
def dicom2dict(x,fin,imp,tags):
    dcm_data = {}
    dcm_data['PatientName'] = str(x.PatientName)
    dt = datetime.strptime(str(x.PatientBirthDate)[:4]+" "+str(x.PatientBirthDate[4:6])+" "+str(x.PatientBirthDate)[6:], '%Y %m %d')
    dcm_data['DOB'] = dt.strftime("%d %B %Y")
    dcm_data['Physician'] = str(x.ReferringPhysicianName)
    dcm_data['Exam'] = str(x.BodyPartExamined)+ " " + str(x.Modality)

    dt = datetime.strptime(str(x.StudyDate)[:4]+" "+str(x.StudyDate[4:6])+" "+str(x.StudyDate)[6:], '%Y %m %d')
    dcm_data['ExamDate'] = dt.strftime("%d %B %Y")
    dcm_data['CH'] = str(x.ReasonForTheRequestedProcedure)
    dcm_data['Findings'] = fin
    dcm_data['Impression'] = imp
    dcm_data['Tags'] = tags
    
    return dcm_data

In [127]:
x = pydicom.dcmread("CXR147_IM-0303-1001.dcm")
#fin,imp,tags = take value from model
dcm_data = dicom2dict(x,fin,imp,tags)
generate_medical_report('CXR147_IM-0303-1001',dcm_data)